In [2]:
import numpy as np
import pandas as pd   
import os
from pathlib import Path
import glob
import json
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import nltk
import cv2
import matplotlib.pyplot as plt
import random
from tensorflow.keras.layers.experimental import preprocessing
import pickle
print('fim')
import time

fim


In [6]:
print(tf.__version__)
from keras.backend import manual_variable_initialization
manual_variable_initialization(True)
start_time = time.clock()

2.4.1


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.


In [8]:
def GravarArquivo ( data_dict,fname):
    fname = fname +"_" +str(len(data_dict)) + '.json'
    print("gravar arquivo: " + fname + " qtd: " +  str(len(data_dict)))
    os.makedirs('Percentage_'+ str(percentageData) , exist_ok=True)
    fname = 'Percentage_'+ str(percentageData) + "/" + fname
    # Create file
    with open(fname, 'w') as outfile:
        json.dump(data_dict, outfile, ensure_ascii=False, indent=4) 
        outfile.close()

In [9]:
def plot_loss(history):
            plt.plot(history.history['loss'], label='training loss')
            plt.plot(history.history['val_loss'], label=' validation loss')
            #plt.ylim([0, 10])
            plt.xlabel('Epopch')
            plt.ylabel('Error')
            plt.legend()
            plt.grid(True)
            plt.savefig( 'Percentage_'+ str(percentageData) + '/error_datalenght.png')
            plt.show()

def plot_acc(history):  
            plt.plot(history.history['sparse_categorical_accuracy'])
            plt.plot(history.history['val_sparse_categorical_accuracy'])
            plt.title('model accuracy')
            plt.ylabel('accuracy')
            plt.xlabel('epoch')
            plt.legend(['train', 'val'], loc='upper left')
            plt.grid(True)
            plt.savefig('Percentage_'+ str(percentageData) + '/accuracy_datalenght.png')
            plt.show()

In [16]:
#Check GPU is available for training or not Or whether the tensorflow version can utilize gpu 
physical_devices = tf.config.list_physical_devices('GPU') 
print("Number of GPUs :", len(physical_devices)) 
print("Tensorflow GPU :",tf.test.is_built_with_cuda())
if len(physical_devices)>0:
    device="/GPU:0"
else:
    device="/CPU:0"
percentageData = 3
posTrainList=[]
posValList=[]
BATCH_SIZE=100
IMG_SIZE=(200,200)
QtdEpocasGravarCHKP = 1
Epochs = 40
lenghtDataTrain = 0
lenghtDataVal =0 
weight = 'weight-050'

Number of GPUs : 0
Tensorflow GPU : True


In [12]:

with open( 'Percentage_'+str(percentageData) +  '/train_dataframe', "rb") as f:
     train_dataframe =  pickle.load(f) 
with open( 'Percentage_'+str(percentageData) +  '/encoder', "rb") as f:
     encoder =  pickle.load(f)
with open( 'Percentage_'+str(percentageData) +  '/val_dataframe', "rb") as f:
     val_dataframe =  pickle.load(f)


In [14]:

#Function that uses the encoder created to encode the input question and answer string
def encode_fn(text):
    return np.array(encoder.encode(text.numpy()))


#Function to load and decode the image from the file paths in the dataframe and use the encoder function
def preprocess(ip,ans):
    img,ques=ip#ip is a list containing image paths and questions
    img=tf.io.read_file(img)
    img=tf.image.decode_jpeg(img,channels=3)
    # quantos canais de cores tem 
    img=tf.image.resize(img,IMG_SIZE)
    img=tf.math.divide(img, 255)# 
    #The question string is converted to encoded list with fixed size of 50 with padding with 0 value
    ques=tf.py_function(encode_fn,inp=[ques],Tout=tf.int32)
    paddings = [[0, 50-tf.shape(ques)[0]]]
    ques = tf.pad(ques, paddings, 'CONSTANT', constant_values=0)
    ques.set_shape([50])#Explicit shape must be defined in order to create the Input pipeline
    
    #The Answer is also encoded 
    ans=tf.py_function(encode_fn,inp=[ans],Tout=tf.int32)
    ans.set_shape([1])
    print(ans)
    return (img,ques),ans
    
def create_pipeline(dataframe):
    raw_df=tf.data.Dataset.from_tensor_slices(((dataframe['Path'],dataframe['Question']),dataframe['Answer']))
    df=raw_df.map(preprocess)#Preprocessing function is applied to the dataset
    df=df.batch(BATCH_SIZE)#The dataset is batched
    return df

#The training and validation Dataset objects are created
train_dataset=create_pipeline(train_dataframe)
validation_dataset=create_pipeline(val_dataframe)

Tensor("EagerPyFunc_1:0", shape=(1,), dtype=int32, device=/job:localhost/replica:0/task:0)
Tensor("EagerPyFunc_1:0", shape=(1,), dtype=int32, device=/job:localhost/replica:0/task:0)


In [17]:
model = tf.keras.models.load_model('Percentage_'+str(percentageData)  +"/ModelTreinamento")



NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for Percentage_3/weight-050

In [19]:
if weight != "":
    model.load_weights('Percentage_'+str(percentageData) +"/" + weight+'.ckpt' )
    model.summary()

print(QtdEpocasGravarCHKP*lenghtDataTrain)
lenghtDataTrain = len(train_dataframe)
print(QtdEpocasGravarCHKP)
print(lenghtDataTrain)
print(BATCH_SIZE)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 256)      26112       text_input[0][0]                 
__________________________________________________________________________________________________
image_input (InputLayer)        [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 50, 512)      1050624     embedding[0][0]                  
______________________________________________________________________________________________

In [20]:
print (len(val_dataframe))
print (len(train_dataframe))



4499
20999


In [ ]:


def scheduler(epoch):
  if epoch < 1:
    return 0.001
  else:
    return 0.001 * tf.math.exp(0.1 * (1 - epoch))

checkpoint_path = 'Percentage_'+ str(percentageData) + '/weights-{epoch:03d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=len(train_dataframe) *QtdEpocasGravarCHKP)

print(QtdEpocasGravarCHKP*lenghtDataTrain)

#LRS = tf.keras.callbacks.LearningRateScheduler(scheduler)
csv_callback=tf.keras.callbacks.CSVLogger(
   'Percentage_'+ str(percentageData) + '/'+ "Training Parameters.csv",
    separator=',', append=False
)


with tf.device(device) :
    history =  model.fit(train_dataset,
              validation_data=validation_dataset,
              callbacks=[csv_callback,cp_callback],
              epochs=Epochs)


hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
history.history
hist.tail()
#plot_loss(history)
#plot_acc(history)


20999
Epoch 1/40


2021-09-06 00:49:49.245303: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-06 00:49:49.246518: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


1050/1050 [==============================] - 1693s 2s/step - loss: 0.4478 - sparse_categorical_accuracy: 0.8208 - val_loss: 6.5568 - val_sparse_categorical_accuracy: 0.3350
Epoch 2/40
1050/1050 [==============================] - 1688s 2s/step - loss: 0.3704 - sparse_categorical_accuracy: 0.8485 - val_loss: 2.3143 - val_sparse_categorical_accuracy: 0.4261
Epoch 3/40
1050/1050 [==============================] - 1773s 2s/step - loss: 0.3463 - sparse_categorical_accuracy: 0.8604 - val_loss: 2.3576 - val_sparse_categorical_accuracy: 0.4225
Epoch 4/40
1050/1050 [==============================] - 1897s 2s/step - loss: 0.3316 - sparse_categorical_accuracy: 0.8674 - val_loss: 2.3732 - val_sparse_categorical_accuracy: 0.4263
Epoch 5/40
1050/1050 [==============================] - 1856s 2s/step - loss: 0.3345 - sparse_categorical_accuracy: 0.8659 - val_loss: 5.6442 - val_sparse_categorical_accuracy: 0.3721
Epoch 6/40
1050/1050 [==============================] - 1828s 2s/step - loss: 0.3405 - spar

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
history.history
hist.tail()
plot_loss(history)
plot_acc(history)

In [ ]:
contadorCerto =0
contadorErro =0 
ArrayQuestoesCertas =[]
ArrarQuestoesErradas=[]

for contador in range(len(train_dataframe)) :
   
    im=cv2.imread(train_dataframe.iloc[contador]['Path'])
    im=cv2.resize(im,(200,200))
    q=train_dataframe.iloc[contador]['Question'] 
    q=encoder.encode(q)
    paddings = [[0, 50-tf.shape(q)[0]]]
    q=tf.pad(q, paddings, 'CONSTANT', constant_values=0)
    q=np.array(q)
    im.resize(1,200,200,3)
    q.resize(1,50)
    ans=model.predict([im,q]) 
    decodAns = encoder.decode([np.argmax(ans)])
        
    
    if train_dataframe.iloc[contador]['Answer'] != decodAns :
        ArrarQuestoesErradas.append ("qestão numero : " + str(contador))
        ArrarQuestoesErradas.append(train_dataframe.iloc[contador]['Question'])
        ArrarQuestoesErradas.append('Repost errada: ' +decodAns + " resp certa:" + train_dataframe.iloc[contador]['Answer']) 
        contadorErro = contadorErro +1
    else:
        ArrayQuestoesCertas.append("qestão numero : " + str(contador))
        ArrayQuestoesCertas.append(train_dataframe.iloc[contador]['Question'])
        ArrayQuestoesCertas.append(  decodAns)
        ArrayQuestoesCertas.append(train_dataframe.iloc[contador]['Answer'])
        contadorCerto = contadorCerto +1

    

In [ ]:
print( "questoes certas" + str(contadorCerto))
print( "questoes erradas" + str(contadorErro))
VerifTrainList=[]
VerifTrainList.append("Acerto - " + str(contadorCerto))
VerifTrainList.append("Erro - " + str(contadorErro ))
GravarArquivo(VerifTrainList,'Verif_Train_resumo' )
GravarArquivo(ArrayQuestoesCertas,'Verif_train_questoescertas' )
GravarArquivo(ArrarQuestoesErradas,'Verif_train_questoeserradas' )


In [ ]:
contadorCerto =0
contadorErro =0 
ArrayQuestoesCertas =[]
ArrarQuestoesErradas=[]

for contador in range(len(val_dataframe)) :
   
    im=cv2.imread(val_dataframe.iloc[contador]['Path'])
    im=cv2.resize(im,(200,200))
    q=val_dataframe.iloc[contador]['Question'] 
    q=encoder.encode(q)
    paddings = [[0, 50-tf.shape(q)[0]]]
    q=tf.pad(q, paddings, 'CONSTANT', constant_values=0)
    q=np.array(q)
    im.resize(1,200,200,3)
    q.resize(1,50)
    ans=model.predict([im,q]) 
    decodAns = encoder.decode([np.argmax(ans)])
    if val_dataframe.iloc[contador]['Answer'] != decodAns :
        ArrarQuestoesErradas.append ("qestão numero : " + str(contador))
        ArrarQuestoesErradas.append(val_dataframe.iloc[contador]['Question'])
        ArrarQuestoesErradas.append( " errado - " + decodAns + " certo - " + val_dataframe.iloc[contador]['Answer'] )
              
        contadorErro = contadorErro +1
    else:
        ArrayQuestoesCertas.append ("qestão numero : " + str(contador))
        ArrayQuestoesCertas.append(val_dataframe.iloc[contador]['Question'])
        ArrayQuestoesCertas.append(decodAns)
        ArrayQuestoesCertas.append(val_dataframe.iloc[contador]['Answer'])
       
        contadorCerto = contadorCerto +1
      

    

In [ ]:
VerifValList=[]
print("certo : " + str(contadorCerto))
print("erro : " + str(contadorErro))
tempo = (time.clock() - start_time, "segundos")
print(tempo)
VerifValList.append("Acerto - " + str(contadorCerto))

VerifValList.append("Erro - " + str(contadorErro ))
VerifValList.append("tempo - " + str(contadorErro ))
GravarArquivo(VerifValList,'Verif_Val_resumo' )
GravarArquivo(ArrayQuestoesCertas,'Verif_val_questoescertas' )
GravarArquivo(ArrarQuestoesErradas,'Verif_val_questoeserradas' )

In [ ]:
tf.keras.models.save_model(model,'Percentage_'+ str(percentageData) +  '/ModelTreinamento_2')

In [ ]:
import pickle

with open('Percentage_'+ str(percentageData) +  '/2train_dataframe', "wb") as f:
    pickle.dump(train_dataframe, f)
with open('Percentage_'+ str(percentageData) +  '/2encoder', "wb") as f:
    pickle.dump(encoder, f)

with open('Percentage_'+ str(percentageData) +  '/2val_dataframe', "wb") as f:
    pickle.dump(val_dataframe, f)
